In [8]:
import numpy as np
import matplotlib.pylab as plt
import scipy.io
from sklearn.model_selection import train_test_split
from tensorflow.keras import models, layers
from tensorflow import keras
from sklearn.metrics import accuracy_score
from keras.utils import to_categorical
import absl.logging
import tensorflow as tf

def mish(x):
    return x * tf.math.tanh(tf.math.softplus(x))

absl.logging.set_verbosity(absl.logging.ERROR) # .h5 에러메세지 무시

mat_file = scipy.io.loadmat("data/PTSDvsHC.mat")

dataset = mat_file['dataset'] 
# 90개의 데이터, 62개의 전극 채널, 1001의 타임스텝 - 1001씩 이미 에포킹을 한 상태이며, 그에따른 평균을 낸 것임(에포킹에 대해)

subject_test_acc = []

X = dataset
y = mat_file['Y']
y = to_categorical(y, num_classes = 2)

X = X.reshape(90, 1001, 62, 1)
X_mean = np.mean(X, axis=2, keepdims=True)

X1 = X
X2 = X_mean

input1_shape = (1001, 62, 1)
input2_shape = (1001, 1, 1)

for i in range(len(dataset)): # loso
    X1_train = np.concatenate([X1[:i], X1[i+1:]], axis=0)
    X1_test = X1[i:i+1]
    X2_train = np.concatenate([X2[:i], X2[i+1:]], axis=0)
    X2_test = X2[i:i+1]
    y_train = np.concatenate([y[:i], y[i+1:]], axis=0)
    y_test = y[i:i+1]
    
    model1_input = layers.Input(shape = input1_shape)
    model2_input = layers.Input(shape = input2_shape)

    branch1_output1 = layers.Conv2D(32, (40, 1))(model1_input) # temporal  filter
    branch1_output1 = layers.BatchNormalization()(branch1_output1)
    branch1_output2 = layers.DepthwiseConv2D((1, 62), padding = 'valid')(branch1_output1) # soatial filter
    branch1_output2 = layers.BatchNormalization()(branch1_output2)
    branch1_output2 = layers.Activation(mish)(branch1_output2)
    branch1_output_end = layers.AveragePooling2D(pool_size = (100, 1), strides = (15, 1))(branch1_output2)
    branch1_output_end = layers.Flatten()(branch1_output_end)

    branch2_output1 = layers.Conv2D(32, (100, 1), strides = (50, 1))(model2_input)
    branch2_output1 = layers.BatchNormalization()(branch2_output1)
    branch2_output1 = layers.Activation(mish)(branch2_output1)
    branch2_output2 = layers.Conv2D(64, (5, 1))(branch2_output1)
    branch2_output2 = layers.BatchNormalization()(branch2_output2)
    branch2_output2 = layers.Activation(mish)(branch2_output2)
    branch2_output_end = layers.GlobalMaxPool2D()(branch2_output2)
    
    combined_input = layers.concatenate([branch1_output_end, branch2_output_end])
    combined_output = layers.Dense(64, activation=mish)(combined_input)
    combined_output = layers.Dropout(0.5)(combined_output)
    
    prediction = layers.Dense(2, activation = 'softmax')(combined_output)
    model = keras.Model(inputs = [model1_input, model2_input], outputs = prediction)
    # model.summary()
    model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
    history = model.fit([X1_train, X2_train], y_train, validation_data = ([X1_test, X2_test], y_test), epochs=100, batch_size=1, verbose=0)
    predictions = model.predict([X1_test, X2_test])
    
    predicted_classes = np.argmax(predictions, axis=1)
    actual_classes = np.argmax(y_test, axis=1)

    acc = accuracy_score(predicted_classes, actual_classes)
    print(f"fold {i} accuracy : {acc}")

    subject_test_acc.append(acc)
print(f"loso mean accuracy : {np.mean(subject_test_acc)}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step
fold 0 accuracy : 1.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step
fold 1 accuracy : 0.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step
fold 2 accuracy : 1.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step
fold 3 accuracy : 1.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step
fold 4 accuracy : 1.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step
fold 5 accuracy : 1.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step
fold 6 accuracy : 0.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step
fold 7 accuracy : 1.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step
fold 8 accuracy : 1.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step
fold 9 accuracy : 1.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
fold 10 accuracy : 1.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step
fold 11 accuracy : 1.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
fold 12 accuracy : 0.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step
fold 13 accuracy : 1.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step
fold 14 accuracy : 0.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
fold 15 accuracy : 0.0
1/1 ━━━━━━━━━━━━━━


KeyboardInterrupt

